# Model Training Notebook

This notebook is designed for training two classifiers: a Convolutional Neural Network (CNN) built from scratch and a pretrained audio model fine-tuned on the BirdCLEF 2025 data. We will load the processed mel spectrogram images, encode the labels, and evaluate the models using accuracy and confusion matrices.

In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

# Import custom scripts
from scripts.utils import load_data, plot_confusion_matrix
from models.cnn_model import create_cnn_model
from models.pretrained_model import load_pretrained_model

# Set paths
mel_spectrograms_dir = 'data/processed/mel_spectrograms/'
labels_dir = 'data/processed/labels/'

# Load data
X, y = load_data(mel_spectrograms_dir, labels_dir)

# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Train the CNN model
cnn_model = create_cnn_model(input_shape=X_train.shape[1:])
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = cnn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate the CNN model
y_pred_cnn = np.argmax(cnn_model.predict(X_test), axis=-1)
cnn_accuracy = accuracy_score(y_test, y_pred_cnn)
print(f'CNN Test Accuracy: {cnn_accuracy:.4f}')

# Confusion matrix for CNN
cm_cnn = confusion_matrix(y_test, y_pred_cnn)
plt.figure(figsize=(10, 7))
plot_confusion_matrix(cm_cnn, classes=np.unique(y), title='CNN Confusion Matrix')

# Load and fine-tune the pretrained model
pretrained_model = load_pretrained_model()
pretrained_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the pretrained model
history_pretrained = pretrained_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate the pretrained model
y_pred_pretrained = np.argmax(pretrained_model.predict(X_test), axis=-1)
pretrained_accuracy = accuracy_score(y_test, y_pred_pretrained)
print(f'Pretrained Model Test Accuracy: {pretrained_accuracy:.4f}')

# Confusion matrix for pretrained model
cm_pretrained = confusion_matrix(y_test, y_pred_pretrained)
plt.figure(figsize=(10, 7))
plot_confusion_matrix(cm_pretrained, classes=np.unique(y), title='Pretrained Model Confusion Matrix')

## Conclusion

In this notebook, we trained a CNN from scratch and fine-tuned a pretrained model on the BirdCLEF 2025 dataset. We evaluated both models using accuracy scores and confusion matrices to understand their performance on the test set.